# Gearoid Lacey C00183380
# Data quality issues and improvements

    #NOTE: The following code assumes the 'Food_Inspections.csv' file is in the same directory as the notebook
The first data quality issue I know exists is different spellings of the same DBA name. To attempt to counteract this I intend to change all DBA names to lowercase and remove any apostrophes, commas full stops and hypens. I will also remove leading and trailing whitespace. I also perform some amendments to the variations of the names mcdonalds, subway, dunkin donuts, 7 eleven and kfc.

Another issue I intend to fix is location data being stored as seperate values and also in a combined string in the location column. From my initial inspections the location data in the location column appears to be more accurate so removing the seperate location rows would appear to be the better option. I initially populate the location column if its empty by creating a tuple and storing the values from the latitude and longitude columns in it. There are also some outliers regarding the location, longitude and latitude rows. There were rows without any location latitude or longitude data but I still keep these rows as theres also an address and zip code which could be used to locate a premises. Note there where no rows missing an address value. Empty cells are populated with the value 'null'

I also noticed that some rows are missing information, therefore I chose to populate these rows with the value null as they are predominatly text based. If they were numerically based then you could potentially fill the missing values with a zero to keep the rows. 

In rows that are missing an AKA Name I will copy in the DBA Name. Or in rows that are missing a DBA Name I copy in the AKA Name. Although the AKA Name is not the legal name of the business copying the AKA Name into the DBA Name may still be useful as they are often quite similar.

As the data is based on premises's in Chicago Illinois, the state column was dropped. 

I noticed that some city values contained misspellings or were populated with names of other cities in Illinois. To counteract this I look for the substring 'chicago' in every city cell. When a city does not contain the chicago substring, I remove the original value and insert 'non-chicago address, attention required' in its place meaning the address or the location column should be used to determine which city the facility is in. Also one city value used was "chcicago". As looking for the substring "chicago" would not work in this case I look specifically for this value also and change it to "chicago".

Also the dates were in the format mm/dd/yyyy. To adjust this I split the date on the occurence of the "/" and reconstruct the date so it is in the format of dd/mm/yyyy. I also allow for dates that contain hyphens instead of forward slashes, if they occur I change them to forward slashes.

When working with the Inspection type column I noticed numerous faults. One of the values present in this column was "two people ate and got sick". To me this is not categorical, therefore I replace that string with "suspected food poisoning" which is another category within this column. Also there were numerous different types of canvass inspections most of which had different spellings of the word canvass. Therefore I ammended each of these spelling mistakes by using the replace function and explicitly stating the error and the replacement value. Another error in the Inspection Type column is when the user performing the inspection appears to leave reminders in the inspection type value e.g 'finish complaint inspection from 5 18 10'. As this Inspection type is in relation to a a complaint I will change the value in the cell to 'complaint'.

I also noticed errors with duplicate license numbers where the license number is 0. As this license number was being assigned to numerous different premises which is incorrect according to the dataset description linked below. I will replace any row with a license number 0 to 'unknown'

Also it is worth noting that the first function my code will run to completion is one that changes every cell to lowercase to avoid discrepencies with upper and lowercase characters.

Regarding duplicates in the csv file, if you open the data in excel and highlight every column except the inspection id column and then press the remove duplicates function in the data tab, it says there are appoximately 87 duplicates. As the inspection ID was different for these, then technically the rows are not duplicated and hence I did not remove them.

Dataset description: https://data.cityofchicago.org/api/assets/BAD5301B-681A-4202-9D25-51B2CAE672FF




In [1]:
%%time
import csv


food = list(csv.DictReader(open('Food_Inspections.csv')))


def fill_cells(data):  # replace empty cells with null value
    result = amend_duplicate_license_number(data)
    for x in result:
        for value in x.keys():
            if x[value] == '':
                x[value] = 'null'
            if len(x['Location']) == 2: #  if location is ('','')
                x['Location'] = 'unknown'
    return result


def amend_duplicate_license_number(data): # license # 0 is duplicated for numerous premises, so change value to unknown
    result = amend_inspections(data)
    for items in result:
        if items['License #'] == '0':
            items['License #'] = 'unknown'
    return result


def amend_inspections(data): # clean inspection type data
    result = amend_duplicate_names(data)
    for items in result:
        items['Inspection Type'] = items['Inspection Type'].replace('canvas', 'canvass')
        items['Inspection Type'] = items['Inspection Type'].replace('canvasss', 'canvass')
        items['Inspection Type'] = items['Inspection Type'].replace('out ofbusiness', 'out of business')
        items['Inspection Type'] = items['Inspection Type'].replace('canvasss for rib fest', 'canvass for rib fest')
        items['Inspection Type'] = items['Inspection Type'].replace('canvasss special event', 'canvass special event')
        items['Inspection Type'] = items['Inspection Type'].replace('fire complain', 'fire complaint')
        items['Inspection Type'] = items['Inspection Type'].replace('two people ate and got sick', 'suspected food poisoning')
        items['Inspection Type'] = items['Inspection Type'].replace('finish complaint inspection from 5 18 10', 'complaint')
    return result


def amend_duplicate_names(data):
    result = remove_duplicate_locations(data)
    for items in result:
        items['DBA Name'] = items['DBA Name'].replace('mc donalds', 'mcdonalds')
        items['DBA Name'] = items['DBA Name'].replace('mcdonalds restaurant', 'mcdonalds')
        items['DBA Name'] = items['DBA Name'].replace('subway sandwiches', 'subway')
        items['DBA Name'] = items['DBA Name'].replace('subway sandwich', 'subway')
        items['DBA Name'] = items['DBA Name'].replace('subway restaurant', 'subway')
        items['DBA Name'] = items['DBA Name'].replace('dunkin donuts   baskin robbins', 'dunkin donuts')
        items['DBA Name'] = items['DBA Name'].replace('dunkin donuts baskin robbins', 'dunkin donuts')
        items['DBA Name'] = items['DBA Name'].replace('kentucky fried chicken', 'kfc')
        items['DBA Name'] = items['DBA Name'].replace('7   eleven', '7 eleven')

    return result


def remove_duplicate_locations(data):  # populate empty locations, then remove longitude and latitude columns
    result = clean_text_data(data)
    temp_data = []
    #  If location col empty populate with longitude and latitude
    for x in result:
        if x['Location'] == '':
            location = (x['Latitude'], x['Longitude'])
            x['Location'] = location
            temp_data.append(x)
        else:
            temp_data.append(x)

    #  Remove longitude and latitude columns
    new_data = []
    for item in temp_data:
        item.pop('Latitude')
        item.pop('Longitude')
        new_data.append(item)
    return new_data


def clean_text_data(data):  # remove punctuation and whitespace to remove some duplicates
    result = amend_city(data)
    text_based_columns = ['DBA Name', 'AKA Name', 'Inspection Type']
    for x in result:
        for item in text_based_columns:
            x[item] = x[item].replace("'", "")
            x[item] = x[item].replace(".", "")
            x[item] = x[item].replace(",", "")
            x[item] = x[item].replace("-", " ")
            x[item] = x[item].replace("/", " ")
            x[item] = x[item].lstrip(" ")
            x[item] = x[item].rstrip(" ")
    return result


def amend_city(data):  # if city value does not contain chicago substring remove the row, then remove the column
    result = remove_state(data)
    new_data = []
    city = 'chicago'
    for x in result:
        if city in x['City'].lower() or x['City'].lower() == '':
            new_data.append(x)
        elif x['City'].lower() == 'chcicago':
            x['City'] = city
        else:
            x['City'] = 'non-chicago address, attention required'
    return new_data


def remove_state(data):
    result = change_date_format(data)
    for x in result:
        x.pop('State')
    return result


def change_date_format(data):  # changes date from mm/dd/yy to dd/mm/yy
    result = fill_names(data)
    for x in result:
        if x['Inspection Date'] != '':
            x['Inspection Date'] = x['Inspection Date'].replace('-', '/')
            temporary_date = x['Inspection Date'].split('/')
            new_date = temporary_date[1] + '/' + temporary_date[0] + '/' + temporary_date[2]
            x['Inspection Date'] = new_date
        if '-' in x['Inspection Date']: # if the dates are separated by a hyphen change to forward slash
            x['Inspection Date'] = x['Inspection Date'].replace('-', '/')
    return result


def fill_names(data):  # populate empty name cells if possible
    result = change_case(data)
    for x in result:
        if x['DBA Name'] == '':
            x['DBA Name'] = x['AKA Name']
        elif x['AKA Name'] == '':
            x['AKA Name'] = x['DBA Name']
    return data


def change_case(data):
    for item in data:
        for value in item:
            item[value] = item[value].lower()
    return data        
    

def write_data(data):
    with open('Output.csv', "w") as fh:
        out_csv = csv.DictWriter(fh, list(data[0].keys()))
        out_csv.writeheader()
        out_csv.writerows(data)


result = fill_cells(food)
write_data(result)
print('Processing finished')


Processing finished
CPU times: user 12.7 s, sys: 593 ms, total: 13.3 s
Wall time: 13.6 s
